In [1]:
import requests
from requests import Response
youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'

res:Response = requests.get(youbike_url)

if res.status_code == 200:
    print("下載成功")
else:
    print("下載失敗")


from pydantic import BaseModel,Field,field_validator,RootModel
class Site (BaseModel):
    站名:str = Field(alias='sna') 
    行政區域:str = Field(alias='sarea')
    時間:str = Field(alias='mday')
    位置:str = Field(alias='ar')
    狀態:bool = Field(alias='act')
    總數量:int = Field(alias='total')
    可借:int = Field(alias='available_rent_bikes')
    緯度:float = Field(alias='latitude')
    經度:float = Field(alias='longitude')     
    可還:int = Field(alias='available_return_bikes')

    @field_validator('站名',mode='before')
    @classmethod
    def get_name(cls,value:str) ->str:
         return value.split('_')[-1]   

class Youbick(RootModel):
    root:list[Site]


data = Youbick.model_validate_json(res.text)
all_sites = data.model_dump()


import pandas as pd
df = pd.DataFrame(all_sites)
df

下載成功


,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
0,捷運科技大樓站,大安區,2024-06-17 19:22:55,復興南路二段235號前,True,28,1,25.02605,121.54360,27
1,復興南路二段273號前,大安區,2024-06-17 19:22:55,復興南路二段273號西側,True,21,9,25.02565,121.54357,12
2,國北教大實小東側門,大安區,2024-06-17 19:06:21,和平東路二段96巷7號,True,16,2,25.02429,121.54124,14
3,和平公園東側,大安區,2024-06-17 19:21:21,和平東路二段118巷33號,True,11,7,25.02351,121.54282,4
4,辛亥復興路口西北側,大安區,2024-06-17 19:22:20,復興南路二段368號,True,16,5,25.02153,121.54299,11
...,...,...,...,...,...,...,...,...,...,...
1418,臺大總圖書館西南側,臺大公館校區,2024-06-17 19:23:20,臺大圖書館西南側,True,30,1,25.01690,121.54031,29
1419,臺大黑森林西側,臺大公館校區,2024-06-17 19:21:21,臺大霖澤館南側,True,20,1,25.01995,121.54347,19
1420,臺大獸醫館南側,臺大公館校區,2024-06-17 19:13:50,臺大獸醫系館南側,True,24,1,25.01791,121.54242,23
1421,臺大新體育館東南側,臺大公館校區,2024-06-17 19:23:19,臺大體育館東側,True,40,9,25.02112,121.53591,31


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1423 entries, 0 to 1422
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   站名      1423 non-null   object 
 1   行政區域    1423 non-null   object 
 2   時間      1423 non-null   object 
 3   位置      1423 non-null   object 
 4   狀態      1423 non-null   bool   
 5   總數量     1423 non-null   int64  
 6   可借      1423 non-null   int64  
 7   緯度      1423 non-null   float64
 8   經度      1423 non-null   float64
 9   可還      1423 non-null   int64  
dtypes: bool(1), float64(2), int64(3), object(4)
memory usage: 101.6+ KB


In [3]:
import numpy as np

less3_df = df.query('可借<=3')
less3_df.shape

(451, 10)

In [4]:
df2 = df.query('可借<=3 and 可還>=30')
df2

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
21,臺灣科技大學正門,大安區,2024-06-17 19:06:22,基隆路四段43號(臺灣科技大學正門旁小側門靠田徑場),True,50,2,25.01451,121.54142,48
22,臺灣科技大學側門,大安區,2024-06-17 19:22:55,基隆路四段73巷與基隆路口,True,99,0,25.01295,121.53973,99
24,臺灣科技大學後門,大安區,2024-06-17 19:23:20,基隆路四段41巷68弄臺科帆船大樓旁,True,39,3,25.01182,121.54165,34
47,嘉興公園,大安區,2024-06-17 19:22:55,樂業街101巷3號西北側,True,44,2,25.02100,121.55211,37
68,敦化基隆路口,大安區,2024-06-17 19:17:30,敦化南路二段218號,True,38,0,25.02215,121.54839,36
...,...,...,...,...,...,...,...,...,...,...
1391,臺大共同教室北側,臺大公館校區,2024-06-17 18:38:21,臺大共同教學館東北側,True,42,1,25.01595,121.53791,41
1399,臺大社科院西側,臺大公館校區,2024-06-17 19:22:20,臺大社科院西側,True,42,3,25.02053,121.54145,39
1400,臺大社會系館南側,臺大公館校區,2024-06-17 19:22:19,臺大社會及社工館南側,True,39,1,25.01987,121.54175,38
1405,臺大生命科學館西北側,臺大公館校區,2024-06-17 19:22:49,臺大動物博物館西側,True,30,0,25.01548,121.53825,30


In [5]:
df2 = df.query('可借<=3 or 可還<=3')
df2

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
0,捷運科技大樓站,大安區,2024-06-17 19:22:55,復興南路二段235號前,True,28,1,25.02605,121.54360,27
2,國北教大實小東側門,大安區,2024-06-17 19:06:21,和平東路二段96巷7號,True,16,2,25.02429,121.54124,14
5,復興南路二段280號前,大安區,2024-06-17 19:05:23,復興南路二段280號,True,11,10,25.02429,121.54328,1
6,復興南路二段340巷口,大安區,2024-06-17 19:17:36,復興南路二段342號,True,11,9,25.02253,121.54326,2
7,新生南路三段52號前,大安區,2024-06-17 19:17:30,新生南路三段52號,True,17,17,25.02112,121.53407,0
...,...,...,...,...,...,...,...,...,...,...
1416,臺大電機二館東南側,臺大公館校區,2024-06-17 19:22:54,臺大電機二館南側,True,14,1,25.01857,121.54247,13
1417,臺大圖資系館北側,臺大公館校區,2024-06-17 19:22:20,臺大圖書資訊館北側,True,25,2,25.01804,121.53928,23
1418,臺大總圖書館西南側,臺大公館校區,2024-06-17 19:23:20,臺大圖書館西南側,True,30,1,25.01690,121.54031,29
1419,臺大黑森林西側,臺大公館校區,2024-06-17 19:21:21,臺大霖澤館南側,True,20,1,25.01995,121.54347,19


In [6]:

df3 = df.query('行政區域=="大安區" and 可借==0')
df3

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
17,溫州停車場,大安區,2024-06-03 14:08:52,新生南路三段54巷/溫州街口,False,5,0,25.020930,121.533100,5
22,臺灣科技大學側門,大安區,2024-06-17 19:22:55,基隆路四段73巷與基隆路口,True,99,0,25.012950,121.539730,99
25,臺大醫學院附設癌醫中心,大安區,2024-06-17 19:17:30,基隆路三段155巷57號西側,True,20,0,25.014260,121.543820,18
28,臺大男七舍前,大安區,2024-06-17 19:22:55,長興街/長興街80巷口西側,True,20,0,25.015370,121.546980,20
31,臺大動物醫院前,大安區,2024-06-17 19:13:15,基隆路三段155巷/基隆路四段口,True,18,0,25.015760,121.542910,18
40,國北教大側門南側,大安區,2024-06-17 19:22:20,敦南街51號,True,25,0,25.022864,121.546622,25
67,敦化南路二段293巷口南側,大安區,2024-06-17 18:58:15,敦化南路二段293巷口南側,True,14,0,25.023700,121.548990,14
68,敦化基隆路口,大安區,2024-06-17 19:17:30,敦化南路二段218號,True,38,0,25.022150,121.548390,36
69,安和路二段181巷口,大安區,2024-06-17 19:16:15,安和路二段185號,True,17,0,25.027210,121.549880,17
76,復興南路1段340巷口,大安區,2024-06-17 19:09:14,復興南路一段信義路三段147巷45弄口,True,50,0,25.037010,121.543480,50


In [7]:
n = 0
df4 = df.query('行政區域=="大安區" and 可借==@n')
df4

,站名,行政區域,時間,位置,狀態,總數量,可借,緯度,經度,可還
17,溫州停車場,大安區,2024-06-03 14:08:52,新生南路三段54巷/溫州街口,False,5,0,25.020930,121.533100,5
22,臺灣科技大學側門,大安區,2024-06-17 19:22:55,基隆路四段73巷與基隆路口,True,99,0,25.012950,121.539730,99
25,臺大醫學院附設癌醫中心,大安區,2024-06-17 19:17:30,基隆路三段155巷57號西側,True,20,0,25.014260,121.543820,18
28,臺大男七舍前,大安區,2024-06-17 19:22:55,長興街/長興街80巷口西側,True,20,0,25.015370,121.546980,20
31,臺大動物醫院前,大安區,2024-06-17 19:13:15,基隆路三段155巷/基隆路四段口,True,18,0,25.015760,121.542910,18
40,國北教大側門南側,大安區,2024-06-17 19:22:20,敦南街51號,True,25,0,25.022864,121.546622,25
67,敦化南路二段293巷口南側,大安區,2024-06-17 18:58:15,敦化南路二段293巷口南側,True,14,0,25.023700,121.548990,14
68,敦化基隆路口,大安區,2024-06-17 19:17:30,敦化南路二段218號,True,38,0,25.022150,121.548390,36
69,安和路二段181巷口,大安區,2024-06-17 19:16:15,安和路二段185號,True,17,0,25.027210,121.549880,17
76,復興南路1段340巷口,大安區,2024-06-17 19:09:14,復興南路一段信義路三段147巷45弄口,True,50,0,25.037010,121.543480,50
